In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(42)
n_samples = 1000
base = np.random.normal(0, 1, size=(n_samples, 1))
X = np.hstack([base + np.random.normal(0, 0.01*i, size=(n_samples,1)) for i in range(1,9)])
true_w = np.array([3.5, -1.2, 0.8, 0, 2.0, 0, -0.5, 1.1])
y = X.dot(true_w) + np.random.normal(0, 0.5, size=n_samples)  # add noise

cols = [f"X{i+1}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=cols)
df['y'] = y
df.head()



,X1,X2,X3,X4,X5,X6,X7,X8,y
0,0.510708,0.483211,0.439480,0.462174,0.475526,0.429869,0.551677,0.494072,3.160602
1,-0.129018,-0.141155,-0.164076,-0.139512,-0.160935,-0.176120,-0.262702,-0.178556,-0.263663
2,0.648285,0.631840,0.635280,0.648409,0.557906,0.591165,0.697721,0.633899,3.732599
3,1.516560,1.516871,1.579660,1.541935,1.506525,1.490150,1.506669,1.580208,7.778142
4,-0.227171,-0.272026,-0.217457,-0.288828,-0.197512,-0.247002,-0.184631,-0.131925,-1.168170


In [5]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[cols])
y = df['y'].values.reshape(-1,1)


X_aug = np.hstack([np.ones((X_scaled.shape[0],1)), X_scaled])  # shape (n, p+1)

def ridge_gd(X, y, alpha=0.01, reg=1.0, n_iter=1000):
    n, p = X.shape
    w = np.zeros((p,1))
    lr = alpha
    costs = []
    for i in range(n_iter):
        preds = X.dot(w)
        error = preds - y
        grad = (2/n)*X.T.dot(error)
        reg_term = (2*reg/n)*w
        reg_term[0] = 0.0
        grad = grad + reg_term
        w = w - lr*grad
        cost = (1/n)*(error**2).sum() + reg*(w[1:]**2).sum()
        costs.append(cost)
    return w, costs
w_test, costs_test = ridge_gd(X_aug, y, alpha=0.01, reg=1.0, n_iter=2000)
print("Final MSE:", costs_test[-1])


Final MSE: 4.230990019407915


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

url = "http://lib.stat.cmu.edu/datasets/boston"
raw = pd.read_csv(url, sep="\s+", skiprows=22, header=None)

data = np.hstack([raw.values[::2, :], raw.values[1::2, :2]])
target = raw.values[1::2, 2]

X = data
y = target

sc = StandardScaler()
X = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = RidgeCV(alphas=[0.1, 1, 10]).fit(X_train, y_train)
lasso = LassoCV(alphas=[0.1, 1, 10]).fit(X_train, y_train)

ridge_pred = ridge.predict(X_test)
lasso_pred = lasso.predict(X_test)

print(ridge.alpha_)
print(lasso.alpha_)
print(r2_score(y_test, ridge_pred))
print(r2_score(y_test, lasso_pred))


<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3245816221.py:9: SyntaxWarning: invalid escape sequence '\s'
  raw = pd.read_csv(url, sep="\s+", skiprows=22, header=None)


10.0
0.1
0.6658059965255931
0.6497587169007897


In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

data = load_iris()
X = data.data
y = data.target

X = np.hstack([np.ones((X.shape[0],1)), X])

m, n = X.shape
classes = np.unique(y)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def train_binary(X, y, lr=0.01, epochs=3000):
    w = np.zeros(n)
    for _ in range(epochs):
        z = X.dot(w)
        h = sigmoid(z)
        grad = (1/m) * X.T.dot(h - y)
        w -= lr * grad
    return w

weights = []
for c in classes:
    y_binary = (y == c).astype(int)
    w = train_binary(X, y_binary)
    weights.append(w)

weights = np.array(weights)

def predict(X):
    scores = X.dot(weights.T)
    return np.argmax(scores, axis=1)

y_pred = predict(X)
print(accuracy_score(y, y_pred))


0.9533333333333334
